In [1]:
import requests # to get the HTML
from bs4 import BeautifulSoup # to get parse the text
import numpy as np
import pandas as pd
import tqdm
from datetime import date, timedelta

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
pd.options.display.max_colwidth = 200

In [3]:
url = 'https://www.jobs.bg/front_job_search.php?frompage=0&add_sh=1&from_hp=1&term=#paging'
#      https://www.jobs.bg/front_job_search.php?frompage=15&add_sh=1&from_hp=1&term=#paging

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [4]:
def get_pages(soup):
    """Get the total number of job offerings and the total number of pages.
    output: total number of pages
    """
    total_job_number = soup.find('td',
                            attrs={'style':
                                  'height:25px;width:220px;font-weight:500;padding-bottom:5px;'}).text.split()[-1]
    total_job_number= int(total_job_number)
    total_pages = int(total_job_number/15)+1 #round up
    return total_pages

In [10]:
# total_pages = get_pages(soup)
total_pages = 30
results =[]

# for page in tqdm.tqdm(range(total_pages)):
for page in tqdm.tqdm(range(2025, 2026)):

    url = f'https://www.jobs.bg/front_job_search.php?frompage={page*15}&add_sh=1&from_hp=1&term=#paging'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    job_infos = soup.find_all("td", attrs = {"class": "offerslistRow"})

    for i in range(0, 15, 3):
        text_info_elem = job_infos[i]
        job_logo_elem = job_infos[i+1]
        job_company_elem = job_infos[i+2]

        job_date = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--overline line-height-1-5'}).text.strip()
        if job_date == 'днес':
            job_date = date.today().strftime("%d.%m.%y")
        elif job_date == 'вчера':
            job_date = (date.today() - timedelta(days=1)).strftime("%d.%m.%y")

        rating_elem = text_info_elem.find('span',attrs={'class':'iconed'})

        if rating_elem:
            rataing_elem_styles = rating_elem["style"].split(";")
            rating_elem_color_styles = [x for x in rataing_elem_styles if "color:" in x]
            rating_color = rating_elem_color_styles[0].split(":")[1]
            rating = rating_elem.text.strip().count("\uf4b3")
        else:
            rating_color = np.nan
            rating = np.nan
        
        
        position = text_info_elem.find('a',attrs={'class':'card__title mdc-typography mdc-typography--headline6 text-overflow'}).text.strip()
        benefits = text_info_elem.find('span',attrs={'class':'card__subtitle mdc-typography mdc-typography--body2 top-margin'}).text.strip()
        results.append([job_date, rating, rating_color, position, benefits])

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  2.60it/s]


In [11]:
jobs_df = pd.DataFrame(results, columns=["date", "rating", "rating_color", "position", "benefits"])
jobs_df.head(2)

,date,rating,rating_color,position,benefits
0,13.04.21,NaN,NaN,Водач ССМ,Пловдив
1,13.04.21,NaN,NaN,Готвач,Русе
